<a href="https://colab.research.google.com/github/knc6/jarvis-tools-notebooks/blob/master/jarvis-tools-notebooks/Cubic_Crystal_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook runs the jupyter-notebook given [here](https://github.com/materialsvirtuallab/m3gnet/blob/main/examples/Cubic%20Crystal%20Test.ipynb) on Colab.

In [ ]:
!pip install m3gnet alignn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import traceback
import warnings

import numpy as np
import pandas as pd
from pymatgen.core import Composition, Lattice, Structure
#from pymatgen.ext.matproj import MPRester
#Run the cell twice to avoid MPRester error
#from pymatgen.ext.matproj import _MPResterLegacy as MPRester
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from tqdm import tqdm

from m3gnet.models import Relaxer

for category in (UserWarning, DeprecationWarning):
    warnings.filterwarnings("ignore", category=category, module="tensorflow")
    warnings.filterwarnings("ignore", category=category, module="pymatgen")

In [ ]:
data = pd.read_html("http://en.wikipedia.org/wiki/Lattice_constant")[0]

In [ ]:
data

,Material,Lattice constant (Å),Crystal structure,Ref.
0,C (diamond),3.567,Diamond (FCC),[7]
1,C (graphite),a = 2.461 c = 6.708,Hexagonal,NaN
2,Si,5.431020511,Diamond (FCC),[8][9]
3,Ge,5.658,Diamond (FCC),[8]
4,AlAs,5.6605,Zinc blende (FCC),[8]
...,...,...,...,...
86,CaVO3,3.767,Cubic perovskite,[15]
87,BaMnO3,a = 5.673 c = 4.71,Hexagonal,[15]
88,CaMnO3,a = 5.27 b = 5.275 c = 7.464,Orthorhombic perovskite,[15]
89,SrRuO3,a = 5.53 b = 5.57 c = 7.85,Orthorhombic perovskite,[15]


In [ ]:

data = data[
    ~data["Crystal structure"].isin(
        ["Hexagonal", "Wurtzite", "Wurtzite (HCP)", "Orthorhombic", "Tetragonal perovskite", "Orthorhombic perovskite"]
    )
]
data.rename(columns={"Lattice constant (Å)": "a (Å)"}, inplace=True)

data["a (Å)"] = data["a (Å)"].map(float)


additional_fcc = """10 Ne 4.43 54 Xe 6.20
13 Al 4.05 58 Ce 5.16
18 Ar 5.26 70 Yb 5.49
20 Ca 5.58 77 Ir 3.84
28 Ni 3.52 78 Pt 3.92
29 Cu 3.61 79 Au 4.08
36 Kr 5.72 82 Pb 4.95
38 Sr 6.08 47 Ag 4.09
45 Rh 3.80 89 Ac 5.31
46 Pd 3.89 90 Th 5.08"""

additional_bcc = """3 Li 3.49 42 Mo 3.15
11 Na 4.23 55 Cs 6.05
19 K 5.23 56 Ba 5.02
23 V 3.02 63 Eu 4.61
24 Cr 2.88 73 Ta 3.31
26 Fe 2.87 74 W 3.16
37 Rb 5.59 41 Nb 3.30"""
def add_new(str_, structure_type, df):
    tokens = str_.split()
    new_crystals = []
    for i in range(int(len(tokens) / 3)):
        el = tokens[3 * i + 1].strip()
        if el not in df["Material"].values:
            new_crystals.append([tokens[3 * i + 1], structure_type, float(tokens[3 * i + 2])])
    df2 = pd.DataFrame(new_crystals, columns=data.columns)
    return pd.concat([df, df2])


# data = add_new(additional_fcc, "FCC", data)
# data = add_new(additional_bcc, "BCC", data)
# data.set_index("Material", inplace=True)
print(data)

       Material     a (Å)  Crystal structure    Ref.
0   C (diamond)  3.567000      Diamond (FCC)     [7]
2            Si  5.431021      Diamond (FCC)  [8][9]
3            Ge  5.658000      Diamond (FCC)     [8]
4          AlAs  5.660500  Zinc blende (FCC)     [8]
5           AlP  5.451000  Zinc blende (FCC)     [8]
..          ...       ...                ...     ...
79        KTaO3  3.988500   Cubic perovskite    [15]
81       SrTiO3  3.988050   Cubic perovskite    [15]
84       EuTiO3  7.810000   Cubic perovskite    [15]
85        SrVO3  3.838000   Cubic perovskite    [15]
86        CaVO3  3.767000   Cubic perovskite    [15]

[73 rows x 4 columns]


<ipython-input-5-08c8062c8bd3>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.rename(columns={"Lattice constant (Å)": "a (Å)"}, inplace=True)
<ipython-input-5-08c8062c8bd3>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["a (Å)"] = data["a (Å)"].map(float)


Setting 5 Angstrom lattice constant to all crystals

In [ ]:
MAPI_KEY_LEGACY = '' #Optional, keep your MAPI_LEGACY_KEY here
predicted = []
mp = []
initial_structs = []
final_m3gnet_structs = []

#mpr = MPRester(MAPI_KEY_LEGACY)

relaxer = Relaxer()  # This loads the default pre-trained model

# warnings.filterwarnings(action="ignore", category=UserWarning, module="tensorflow")
currently_m3gnet_failing = ['CsCl','CsI','Fe','Mo','W','V','Nb','Ta','Ne','CaVO3','Xe','Ar','Yb','Kr','Sr','Ac','Li','Na','Cs','K','Ba','Eu','Cr','Rb',"NC0.99"]

#for formula, v in tqdm(data.iterrows(), total=len(data)):
for ii,i in tqdm(data.iterrows(), total=len(data)):
  #M3gnet failing so skipping these
  formula=i['Material']
  if formula not in currently_m3gnet_failing:
    formula = formula.split()[0]
    c = Composition(formula)
    els = sorted(c.elements)
    #cs = v["Crystal structure"]
    cs = i["Crystal structure"]
    wiki = i['a (Å)']
    # We initialize all the crystals with an arbitrary lattice constant of 5 angstroms.
    if "Zinc blende" in cs:
        s = Structure.from_spacegroup("F-43m", Lattice.cubic(5), [els[0], els[1]], [[0, 0, 0], [0.25, 0.25, 0.75]])
    elif "Halite" in cs:
        s = Structure.from_spacegroup("Fm-3m", Lattice.cubic(5), [els[0], els[1]], [[0, 0, 0], [0.5, 0, 0]])
    elif "Caesium chloride" in cs:
        s = Structure.from_spacegroup("Pm-3m", Lattice.cubic(5), [els[0], els[1]], [[0, 0, 0], [0.5, 0.5, 0.5]])
    elif "Cubic perovskite" in cs:
        s = Structure(
            Lattice.cubic(5),
            [els[0], els[1], els[2], els[2], els[2]],
            [[0.0, 0.0, 0.0], [0.5, 0.5, 0.5], [0.5, 0.5, 0], [0.0, 0.5, 0.5], [0.5, 0, 0.5]],
        )
    elif "Diamond" in cs:
        s = Structure.from_spacegroup("Fd-3m", Lattice.cubic(5), [els[0]], [[0.25, 0.75, 0.25]])
    elif "BCC" in cs:
        s = Structure(Lattice.cubic(5), [els[0]] * 2, [[0.0, 0.0, 0.0], [0.5, 0.5, 0.5]])
    elif "FCC" in cs:
        s = Structure(Lattice.cubic(5), [els[0]] * 4, [[0.0, 0.0, 0.0], [0.5, 0.5, 0], [0.0, 0.5, 0.5], [0.5, 0, 0.5]])
    else:
        predicted.append(0)
        mp.append(0)
        continue
    print(s)
    initial_structs.append(s)
    relax_results = relaxer.relax(s)

    final_structure = relax_results["final_structure"]
    final_m3gnet_structs.append(final_structure)
    predicted.append(final_structure.lattice.a)

    #break
    try:
        mids = mpr.get_entries(s.composition.reduced_formula)
        for i in mids:
            try:
                structure = mpr.get_structure_by_material_id(i.entry_id)
                sga = SpacegroupAnalyzer(structure)
                sga2 = SpacegroupAnalyzer(final_structure)
                if sga.get_space_group_number() == sga2.get_space_group_number():
                    conv = sga.get_conventional_standard_structure()
                    mp.append(conv.lattice.a)
                    break
            except Exception:
                pass
        else:
            raise RuntimeError
    except Exception:
        mp.append(0)
        traceback.print_exc()
    print('\nInitial,final,MP,Wiki,formula',5,final_structure.lattice.a,mp[-1],wiki)

  0%|          | 0/73 [00:00<?, ?it/s]

Full Formula (C8)
Reduced Formula: C
abc   :   5.000000   5.000000   5.000000
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (8)
  #  SP       a     b     c
---  ----  ----  ----  ----
  0  C     0.25  0.25  0.75
  1  C     0     0.5   0
  2  C     0.75  0.75  0.75
  3  C     0.5   0.5   0.5
  4  C     0.75  0.25  0.25
  5  C     0.5   0     0
  6  C     0     0     0.5
  7  C     0.25  0.75  0.25


Traceback (most recent call last):
  File "<ipython-input-6-a9bc7b7bd966>", line 58, in <cell line: 15>
    mids = mpr.get_entries(s.composition.reduced_formula)
NameError: name 'mpr' is not defined
  1%|▏         | 1/73 [00:14<17:47, 14.82s/it]


Initial,final,MP,Wiki,formula 5 3.5718039756536863 0 3.567
Full Formula (Si8)
Reduced Formula: Si
abc   :   5.000000   5.000000   5.000000
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (8)
  #  SP       a     b     c
---  ----  ----  ----  ----
  0  Si    0.25  0.25  0.75
  1  Si    0     0.5   0
  2  Si    0.75  0.75  0.75
  3  Si    0.5   0.5   0.5
  4  Si    0.75  0.25  0.25
  5  Si    0.5   0     0
  6  Si    0     0     0.5
  7  Si    0.25  0.75  0.25


Traceback (most recent call last):
  File "<ipython-input-6-a9bc7b7bd966>", line 58, in <cell line: 15>
    mids = mpr.get_entries(s.composition.reduced_formula)
NameError: name 'mpr' is not defined
  3%|▎         | 2/73 [00:15<07:29,  6.33s/it]

...


Initial,final,MP,Wiki,formula 5 3.9448066067159178 0 3.98805
Full Formula (Eu1 Ti1 O3)
Reduced Formula: EuTiO3
abc   :   5.000000   5.000000   5.000000
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (5)
  #  SP      a    b    c
---  ----  ---  ---  ---
  0  Eu    0    0    0
  1  Ti    0.5  0.5  0.5
  2  O     0.5  0.5  0
  3  O     0    0.5  0.5
  4  O     0.5  0    0.5


Traceback (most recent call last):
  File "<ipython-input-6-a9bc7b7bd966>", line 58, in <cell line: 15>
    mids = mpr.get_entries(s.composition.reduced_formula)
NameError: name 'mpr' is not defined
 97%|█████████▋| 71/73 [01:06<00:01,  1.23it/s]


Initial,final,MP,Wiki,formula 5 3.9294272293452503 0 7.81
Full Formula (Sr1 V1 O3)
Reduced Formula: SrVO3
abc   :   5.000000   5.000000   5.000000
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (5)
  #  SP      a    b    c
---  ----  ---  ---  ---
  0  Sr    0    0    0
  1  V     0.5  0.5  0.5
  2  O     0.5  0.5  0
  3  O     0    0.5  0.5
  4  O     0.5  0    0.5


Traceback (most recent call last):
  File "<ipython-input-6-a9bc7b7bd966>", line 58, in <cell line: 15>
    mids = mpr.get_entries(s.composition.reduced_formula)
NameError: name 'mpr' is not defined
100%|██████████| 73/73 [01:07<00:00,  1.07it/s]


Initial,final,MP,Wiki,formula 5 3.906043550979565 0 3.838


In [ ]:
data_1 = data[~data["Material"].isin(currently_m3gnet_failing)]

In [ ]:
data_1

,Material,a (Å),Crystal structure,Ref.
0,C (diamond),3.567000,Diamond (FCC),[7]
2,Si,5.431021,Diamond (FCC),[8][9]
3,Ge,5.658000,Diamond (FCC),[8]
4,AlAs,5.660500,Zinc blende (FCC),[8]
5,AlP,5.451000,Zinc blende (FCC),[8]
...,...,...,...,...
77,ScN,4.520000,Halite,[14]
79,KTaO3,3.988500,Cubic perovskite,[15]
81,SrTiO3,3.988050,Cubic perovskite,[15]
84,EuTiO3,7.810000,Cubic perovskite,[15]


In [ ]:
data_1["MP a (Å)"] = mp
data_1["Predicted a (Å)"] = predicted
data_1["initial_structs"] = initial_structs
data_1["final_m3gnet_structs"] = final_m3gnet_structs

<ipython-input-9-02a3f8920e94>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_1["MP a (Å)"] = mp
<ipython-input-9-02a3f8920e94>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_1["Predicted a (Å)"] = predicted
<ipython-input-9-02a3f8920e94>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html

In [ ]:
mem=[]
for i,ii in data_1.iterrows():
    info={}
    info['material']=ii['Material']
    info['a'] = ii['a (Å)']
    info['Crystal structure']=ii['Crystal structure']
    info['atoms']=(pmg_to_atoms(ii['initial_structs'])).to_dict()
    info['ref']=ii['Ref.']
    #print(info)
    mem.append(info)
from jarvis.db.jsonutils import dumpjson
dumpjson(data=mem,filename='data_1.json')

In [ ]:
from sklearn.metrics import mean_absolute_error
print('MAE wrt Exp/Wiki data',mean_absolute_error(data_1['a (Å)'],data_1['Predicted a (Å)']))
print('MAE wrt MP data',mean_absolute_error(data_1['MP a (Å)'],data_1['Predicted a (Å)']))

MAE wrt Exp/Wiki data 0.13163907193121901
MAE wrt MP data 5.175435282714427


In [ ]:
from jarvis.core.atoms import Atoms
from alignn.ff.ff import ForceField, default_path
from jarvis.core.atoms import pmg_to_atoms
model_path = default_path()
def optimize_atoms(atoms=[],model_path=model_path):
    atoms = pmg_to_atoms(atoms)
    print(atoms.composition.reduced_formula)
    ff = ForceField(
        jarvis_atoms=atoms,
        model_path=model_path,
        stress_wt=.3,
        force_mult_natoms=False
    )
    opt, en, fs = ff.optimize_atoms()
    return opt.lattice.a

model_path /usr/local/lib/python3.10/dist-packages/alignn/ff


In [ ]:
tqdm.pandas()
#slow
data_1['alignn_a']=data_1['initial_structs'].progress_apply(lambda x:optimize_atoms(atoms=x))

  0%|          | 0/63 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/dgl/backend/pytorch/tensor.py:445: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), (


C
OPTIMIZATION
a= 5.000 Ang b= 5.000 Ang c= 5.000 Ang Volume= 125.000 amu/a3 PE=-17.14334 eV KE= 0.00000 eV T= 0.000 K 
a= 4.963 Ang b= 4.963 Ang c= 4.963 Ang Volume= 122.275 amu/a3 PE=-17.65818 eV KE= 0.00000 eV T= 0.000 K 
a= 4.890 Ang b= 4.890 Ang c= 4.890 Ang Volume= 116.919 amu/a3 PE=-18.59226 eV KE= 0.00000 eV T= 0.000 K 
a= 4.779 Ang b= 4.778 Ang c= 4.778 Ang Volume= 109.102 amu/a3 PE=-20.27380 eV KE= 0.00000 eV T= 0.000 K 
a= 4.624 Ang b= 4.620 Ang c= 4.620 Ang Volume= 98.672 amu/a3 PE=-23.85551 eV KE= 0.00000 eV T= 0.000 K 
a= 4.406 Ang b= 4.392 Ang c= 4.392 Ang Volume= 84.978 amu/a3 PE=-30.91432 eV KE= 0.00000 eV T= 0.000 K 
a= 4.144 Ang b= 4.108 Ang c= 4.108 Ang Volume= 69.911 amu/a3 PE=-36.85871 eV KE= 0.00000 eV T= 0.000 K 
a= 3.848 Ang b= 3.792 Ang c= 3.792 Ang Volume= 55.299 amu/a3 PE=-58.93035 eV KE= 0.00000 eV T= 0.000 K 
a= 3.465 Ang b= 3.391 Ang c= 3.391 Ang Volume= 39.738 amu/a3 PE=-61.08579 eV KE= 0.00000 eV T= 0.000 K 
a= 3.466 Ang b= 3.392 Ang c= 3.392 Ang Volume

  3%|▎         | 2/63 [01:10<36:05, 35.49s/it]

Si
OPTIMIZATION
a= 5.000 Ang b= 5.000 Ang c= 5.000 Ang Volume= 125.000 amu/a3 PE=-24.25818 eV KE= 0.00000 eV T= 0.000 K 
a= 5.430 Ang b= 5.430 Ang c= 5.430 Ang Volume= 160.108 amu/a3 PE=-32.73911 eV KE= 0.00000 eV T= 0.000 K 
a= 5.428 Ang b= 5.428 Ang c= 5.428 Ang Volume= 159.930 amu/a3 PE=-32.73807 eV KE= 0.00000 eV T= 0.000 K 
a= 5.424 Ang b= 5.424 Ang c= 5.424 Ang Volume= 159.577 amu/a3 PE=-32.74663 eV KE= 0.00000 eV T= 0.000 K 
a= 5.418 Ang b= 5.417 Ang c= 5.418 Ang Volume= 159.053 amu/a3 PE=-32.75916 eV KE= 0.00000 eV T= 0.000 K 
a= 5.411 Ang b= 5.410 Ang c= 5.410 Ang Volume= 158.364 amu/a3 PE=-32.77507 eV KE= 0.00000 eV T= 0.000 K 
a= 5.400 Ang b= 5.401 Ang c= 5.401 Ang Volume= 157.517 amu/a3 PE=-32.79468 eV KE= 0.00000 eV T= 0.000 K 
a= 5.389 Ang b= 5.390 Ang c= 5.389 Ang Volume= 156.523 amu/a3 PE=-32.81676 eV KE= 0.00000 eV T= 0.000 K 
a= 5.376 Ang b= 5.376 Ang c= 5.377 Ang Volume= 155.388 amu/a3 PE=-32.84114 eV KE= 0.00000 eV T= 0.000 K 
a= 5.360 Ang b= 5.359 Ang c= 5.361 Ang 

  5%|▍         | 3/63 [02:22<50:40, 50.68s/it]

...

TiN
OPTIMIZATION
a= 5.000 Ang b= 5.000 Ang c= 5.000 Ang Volume= 125.000 amu/a3 PE=-43.43607 eV KE= 0.00000 eV T= 0.000 K 
a= 4.872 Ang b= 4.872 Ang c= 4.872 Ang Volume= 115.643 amu/a3 PE=-52.87354 eV KE= 0.00000 eV T= 0.000 K 
a= 4.655 Ang b= 4.652 Ang c= 4.649 Ang Volume= 100.662 amu/a3 PE=-58.89113 eV KE= 0.00000 eV T= 0.000 K 
a= 4.398 Ang b= 4.391 Ang c= 4.385 Ang Volume= 84.685 amu/a3 PE=-62.08990 eV KE= 0.00000 eV T= 0.000 K 
a= 4.143 Ang b= 4.131 Ang c= 4.119 Ang Volume= 70.499 amu/a3 PE=-62.52551 eV KE= 0.00000 eV T= 0.000 K 
a= 4.144 Ang b= 4.132 Ang c= 4.121 Ang Volume= 70.572 amu/a3 PE=-62.53094 eV KE= 0.00000 eV T= 0.000 K 
a= 4.145 Ang b= 4.135 Ang c= 4.126 Ang Volume= 70.717 amu/a3 PE=-62.54139 eV KE= 0.00000 eV T= 0.000 K 
a= 4.147 Ang b= 4.139 Ang c= 4.132 Ang Volume= 70.932 amu/a3 PE=-62.55621 eV KE= 0.00000 eV T= 0.000 K 
a= 4.149 Ang b= 4.145 Ang c= 4.141 Ang Volume= 71.212 amu/a3 PE=-62.57431 eV KE= 0.00000 eV T= 0.000 K 
a= 4.152 Ang b= 4.151 Ang c= 4.152 Ang Volum

 78%|███████▊  | 49/63 [2:20:47<10:02, 43.05s/it]

ZrN
OPTIMIZATION
a= 5.000 Ang b= 5.000 Ang c= 5.000 Ang Volume= 125.000 amu/a3 PE=-60.07904 eV KE= 0.00000 eV T= 0.000 K 
a= 4.959 Ang b= 4.959 Ang c= 4.959 Ang Volume= 121.985 amu/a3 PE=-62.82177 eV KE= 0.00000 eV T= 0.000 K 
a= 4.887 Ang b= 4.888 Ang c= 4.886 Ang Volume= 116.707 amu/a3 PE=-63.49846 eV KE= 0.00000 eV T= 0.000 K 
a= 4.791 Ang b= 4.794 Ang c= 4.788 Ang Volume= 110.001 amu/a3 PE=-64.17685 eV KE= 0.00000 eV T= 0.000 K 
a= 4.680 Ang b= 4.685 Ang c= 4.676 Ang Volume= 102.534 amu/a3 PE=-64.75758 eV KE= 0.00000 eV T= 0.000 K 
a= 4.561 Ang b= 4.565 Ang c= 4.556 Ang Volume= 94.871 amu/a3 PE=-65.10579 eV KE= 0.00000 eV T= 0.000 K 
a= 4.444 Ang b= 4.438 Ang c= 4.449 Ang Volume= 87.754 amu/a3 PE=-65.10481 eV KE= 0.00000 eV T= 0.000 K 
a= 4.444 Ang b= 4.440 Ang c= 4.449 Ang Volume= 87.801 amu/a3 PE=-65.10880 eV KE= 0.00000 eV T= 0.000 K 
a= 4.446 Ang b= 4.444 Ang c= 4.448 Ang Volume= 87.893 amu/a3 PE=-65.11665 eV KE= 0.00000 eV T= 0.000 K 
a= 4.448 Ang b= 4.450 Ang c= 4.447 Ang Vol

 79%|███████▉  | 50/63 [3:11:38<3:24:48, 945.24s/it]

HfN
OPTIMIZATION
a= 5.000 Ang b= 5.000 Ang c= 5.000 Ang Volume= 125.000 amu/a3 PE=-66.84406 eV KE= 0.00000 eV T= 0.000 K 
a= 4.968 Ang b= 4.968 Ang c= 4.968 Ang Volume= 122.624 amu/a3 PE=-68.68942 eV KE= 0.00000 eV T= 0.000 K 
a= 4.903 Ang b= 4.901 Ang c= 4.899 Ang Volume= 117.740 amu/a3 PE=-69.35104 eV KE= 0.00000 eV T= 0.000 K 
a= 4.815 Ang b= 4.810 Ang c= 4.806 Ang Volume= 111.309 amu/a3 PE=-69.99635 eV KE= 0.00000 eV T= 0.000 K 
a= 4.712 Ang b= 4.704 Ang c= 4.696 Ang Volume= 104.100 amu/a3 PE=-70.54527 eV KE= 0.00000 eV T= 0.000 K 
a= 4.598 Ang b= 4.588 Ang c= 4.579 Ang Volume= 96.595 amu/a3 PE=-70.95288 eV KE= 0.00000 eV T= 0.000 K 
a= 4.476 Ang b= 4.470 Ang c= 4.464 Ang Volume= 89.310 amu/a3 PE=-71.07629 eV KE= 0.00000 eV T= 0.000 K 
a= 4.475 Ang b= 4.470 Ang c= 4.466 Ang Volume= 89.328 amu/a3 PE=-71.07887 eV KE= 0.00000 eV T= 0.000 K 
a= 4.474 Ang b= 4.471 Ang c= 4.468 Ang Volume= 89.363 amu/a3 PE=-71.08395 eV KE= 0.00000 eV T= 0.000 K 
a= 4.471 Ang b= 4.472 Ang c= 4.472 Ang Vol

In [ ]:
pip freeze